In [1]:
!pip install python-dotenv
!git init
!touch .gitignore
!echo "info.env" >> .gitignore
!git check-ignore -v .env


hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/


In [61]:
import pandas as pd
import http.client
import json
import requests
import os
from dotenv import load_dotenv
import re

# Load environment variables
load_dotenv("info.env")

# Access credentials
authUsername = os.getenv("API_USERNAME")
authPassword = os.getenv("API_PASSWORD")

### Pre-filtering based on inventory ###
# Search based on specific filters
page = 8
page_size = 20
craft = 'crochet'
knit_gauge = 5
weight = 'DK'
query = 'dragon'

# Define URL for the API request
url = 'https://api.ravelry.com/patterns/search.json?page={}&page_size={}&craft={}'.format(page, page_size, craft)
# Make request
r = requests.get(url, auth=requests.auth.HTTPBasicAuth(authUsername, authPassword))
# Close connection
r.close()

def getPatterns():
  yarndf = pd.DataFrame()
  keydf = pd.DataFrame()
  if r.status_code == 200:
      data = r.json()
      # Extract pattern ID(s) from the search results
      if 'patterns' in data and len(data['patterns']) > 0:
        for i in range(len(data['patterns'])):
            pattern_id = data['patterns'][i]['id']

            # Define URL to get pattern details
            pattern_url = f'https://api.ravelry.com/patterns/{pattern_id}.json'

            # Make the request for pattern details
            pattern_response = requests.get(pattern_url, auth=requests.auth.HTTPBasicAuth(authUsername, authPassword))
            pattern_response.close()

            # If patterns found, collect the necessary yarn info
            if pattern_response.status_code == 200:
                pattern_data = pattern_response.json()
                try:
                  #print(json.dumps(json.loads(pattern_response.text), indent=4))
                  yarnData = pattern_data['pattern']['yarn_weight']

                  # Extract yardage for minimum/maximums
                  yardage = ['0', '0']
                  extracted = re.findall(r'\d+', pattern_data['pattern']['yardage_description'])

                  if len(extracted) == 2:
                    yardage = extracted
                  elif len(extracted) == 1:
                    yardage[0] = extracted[0]
                    yardage[1] = extracted[0]

                  # Get project name and add all to dataframe
                  yarndf = pd.concat([yarndf, pd.DataFrame([{'Project name': pattern_data['pattern']['name'],
                                                            'Yarn id': yarnData['id'],
                                                            'Crochet gauge': yarnData['crochet_gauge'],
                                                            'Knit gauge': yarnData['knit_gauge'],
                                                            'Yarn name': yarnData['name'],
                                                            'ply': yarnData['ply'], 'wpi': yarnData['wpi'],
                                                            'Min yardage': int(yardage[0]),
                                                            'Max yardage': int(yardage[1]),
                                                            'Notes' : pattern_data['pattern']['notes']}])], ignore_index=True)

                  # Get key information from pattern details for recommender
                  keydf = pd.concat([keydf, pd.DataFrame([{'Project name': pattern_data['pattern']['name'],
                                                          'Project id': pattern_data['pattern']['id'],
                                                          'Difficulty average': pattern_data['pattern']['difficulty_average'],
                                                          'UK': pattern_data['pattern']['has_uk_terminology'],
                                                          'US' : pattern_data['pattern']['has_us_terminology']}])], ignore_index=True)
                except:
                  print("No yarn weight recorded")
      else:
        print("No patterns found")
  else:
    print("Unable to access patterns")

  keydf = keydf.merge(yarndf, on='Project name')
  print(keydf)
  return keydf


def fetchInventory():
  # Open stored inventory into dataframe
  with open("inventory.json", "r") as file:
    inventory = json.load(file)
    inv = pd.DataFrame(inventory)
    groupedInv = inv.groupby(['Yarn name','ply', 'wpi', 'Total yardage']).agg(
        min_yardage=('Total yardage', 'min'),
        max_yardage=('Total yardage', 'max')).reset_index()

    print(groupedInv)
    return groupedInv

def filterPatterns(groupedInv, patterns):
  # Filters patterns for yarn available in inventory
  filtereddf = patterns[patterns['Yarn name'].isin(groupedInv['Yarn name'])]
  finaldf = pd.DataFrame(columns = patterns.columns)
  for idx, row in filtereddf.iterrows():
    rowYardage = row['Max yardage']
    maxYardage = int(max(groupedInv.loc[groupedInv['Yarn name'] == row['Yarn name'], 'max_yardage'].values))

    if (rowYardage > 0) & (rowYardage <= maxYardage):
      # Directly adds if empty for future compatibility
      if finaldf.empty:
        finaldf = pd.DataFrame([row])
      else:
        finaldf = pd.concat([finaldf, pd.DataFrame([row])], ignore_index = True)

  print(finaldf)

patterns = getPatterns()


                               Project name  Project id  Difficulty average  \
0                                    Freyja     7408440            1.666667   
1   Double Petal Daisy Flower Granny Square     7414294            0.000000   
2                  Simple Twisted Earwarmer      980510            1.637795   
3                        Anchor the Octopus     7343415            3.733333   
4                        Battenberg Blanket      885079            2.247273   
5                       Jessie Stash Basket     1113818            2.575163   
6                         Fred the Dinosaur     1151249            2.640000   
7                                  Mosu Bag     1250661            2.275362   
8                      Brain Eater Fish Hat     1344796            0.000000   
9                                Pampa Cowl     7399067            4.941176   
10                                  Babylub     7401150            2.571429   
11                           Granny Stripes      184

In [70]:
# Text pre-processing before recommender
chars = r'[^\w\s]' # Code for removing special characters
patterns['Notes'] = patterns['Notes'].replace(chars, '', regex=True)
patterns['Project name'] = patterns['Project name'].replace(chars, '', regex=True)

# Get current yarn inventory and filter through patterns
groupedInv = fetchInventory()
filterPatterns(groupedInv, patterns)


   Yarn name  ply  wpi  Total yardage  min_yardage  max_yardage
0         DK    8   11            500          500          500
1         DK    8   12            200          200          200
2  Fingering    4   14            800          800          800
               Project name  Project id  Difficulty average     UK    US  \
0  Simple Twisted Earwarmer      980510            1.637795  False  True   
1                Pampa Cowl     7399067            4.941176   None  True   
2       Granny Stripe Shawl     7370418            2.380952   None  True   

   Yarn id Crochet gauge Knit gauge  Yarn name ply wpi  Min yardage  \
0       11          None         22         DK   8  11           80   
1       11          None         22         DK   8  11          245   
2        5                       28  Fingering   4  14          273   

   Max yardage                                              Notes  
0          120  There are so many reasons I love earwarmers I ...  
1          328    

In [69]:
### Basic recommender ###
# Take the last 5 patterns saved by the user and use the average calculated metrics to then recommend
# In release, should use the saved and completed patterns
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity


def readSaved():
  with open("saved.json", "r") as file:
    savedPatterns = json.load(file)
    saved = pd.DataFrame(savedPatterns)
    return saved

def vectorize(names, bow, vectorizer):
  tfidfTransformer = TfidfTransformer()
  tfidfMatrix = tfidfTransformer.fit_transform(bow)
  columnNames = vectorizer.get_feature_names_out() # Gets words for column naming

  return pd.DataFrame(tfidfMatrix.toarray(), index=names, columns = columnNames)

def transformPatterns(saved, patterns):
  savedNames = saved['Project name'].tolist()
  #savedNames += sum(saved['Notes'].str.split(), [])
  vectorizer = CountVectorizer(stop_words='english') #Removes stop words to reduce unnecessary processing
  bagOfWordsMatrix = vectorizer.fit_transform(savedNames)
  one = vectorize(savedNames,bagOfWordsMatrix, vectorizer)

  patternNames = patterns['Project name'].tolist()
  bagOfWordsMatrix = vectorizer.transform(patternNames) # Maps to the saved pattern words
  two = vectorize(patternNames, bagOfWordsMatrix, vectorizer)
  similar = cosine_similarity(two, one)
  print(pd.DataFrame(similar, index=patternNames, columns=savedNames))


saved = readSaved()
transformPatterns(saved, patterns)




                                         Dante the Dragon  \
Freyja                                                0.0   
Double Petal Daisy Flower Granny Square               0.0   
Simple Twisted Earwarmer                              0.0   
Anchor the Octopus                                    0.0   
Battenberg Blanket                                    0.0   
Jessie Stash Basket                                   0.0   
Fred the Dinosaur                                     0.0   
Mosu Bag                                              0.0   
Brain Eater Fish Hat                                  0.0   
Pampa Cowl                                            0.0   
Babylub                                               0.0   
Granny Stripes                                        0.0   
Moss Stitch in a Square Blanket                       0.0   
Dumpster Fire amigurumi                               0.0   
Granny Stripe Shawl                                   0.0   
The Original Granny Mush

Notes for retrieving names for the recommender
*   Remove any symbols from pattern names cause having " causes issues with the json
*   Any missing data should be 0 or none
* Probably include links to the projects when saving to the json


